# A Simple Neural Network from Scratch with PyTorch and Google Colab

In this tutorial we implement a simple neural network from scratch using PyTorch.


## About

In this tutorial we will implement a simple neural network from scratch using PyTorch. The idea of the tutorial is to teach you the basics of PyTorch and how it can be used to implement a neural network from scratch. I will go over some of the basic functionalities and concepts available in PyTorch that will allow you to build your own neural networks. 

This tutorial assumes you have prior knowledge of how a neural network works. Don’t worry! Even if you are not so sure, you will be okay. For advanced PyTorch users, this tutorial may still serve as a refresher. This tutorial is heavily inspired by this [Neural Network implementation](https://repl.it/talk/announcements/Build-a-Neural-Network-in-Python/5457) coded purely using Numpy. In fact, I tried re-implementing the code using PyTorch instead and added my own intuitions and explanations. Thanks to [Samay](https://repl.it/@shamdasani) for his phenomenal work, I hope this inspires many others as it did with me.

The `torch` module provides all the necessary **tensor** operators you will need to implement your first neural network from scratch in PyTorch. That's right! In PyTorch everything is a Tensor, so this is the first thing you will need to get used to. Let's import the libraries we will need for this tutorial.

In [1]:
import torch#安装torch库，支持在图形处理单元上计算张量
import torch.nn as nn#加载神经网络的常用模块

## Data
Let's start by creating some sample data using the `torch.tensor` command. In Numpy, this could be done with `np.array`. Both functions serve the same purpose, but in PyTorch everything is a Tensor as opposed to a vector or matrix. We define types in PyTorch using the `dtype=torch.xxx` command. 

In the data below, `X` represents the amount of hours studied and how much time students spent sleeping, whereas `y` represent grades. The variable `xPredicted` is a single input for which we want to predict a grade using the parameters learned by the neural network. Remember, the neural network wants to learn a mapping between `X` and `y`, so it will try to take a guess from what it has learned from the training data. 

In [2]:
X = torch.tensor(([2, 9], [1, 5], [3, 6]), dtype=torch.float) #定义张量，大小为3X2，浮点型
y = torch.tensor(([92], [100], [89]), dtype=torch.float) #定义张量，大小为3X1，浮点型
xPredicted = torch.tensor(([4, 8]), dtype=torch.float) #定义张量，大小为1X2，浮点型

You can check the size of the tensors we have just created with the `size` command. This is equivalent to the `shape` command used in tools such as Numpy and Tensorflow. 

In [3]:
print(X.size())#输出x,y张量大小
print(y.size())

torch.Size([3, 2])
torch.Size([3, 1])


## Scaling

Below we are performing some scaling on the sample data. Notice that the `max` function returns both a tensor and the corresponding indices. So we use `_` to capture the indices which we won't use here because we are only interested in the max values to conduct the scaling. Perfect! Our data is now in a very nice format our neural network will appreciate later on. 

In [4]:
# 标度单位
X_max, _ = torch.max(X, 0)#返回X中每一列中最大值的那个元素的索引
xPredicted_max, _ = torch.max(xPredicted, 0)#返回xPredicted中每一列中最大值的那个元素的索引

X = torch.div(X, X_max)#对X和X每列最大值作除法运算
xPredicted = torch.div(xPredicted, xPredicted_max)#对xPredicted和xPredicted每列最大值作除法运算
y = y / 100  # 最高测试分数为100
print(xPredicted)#打印结果

tensor([0.5000, 1.0000])


Notice that there are two functions `max` and `div` that I didn't discuss above. They do exactly what they imply: `max` finds the maximum value in a vector... I mean tensor; and `div` is basically a nice little function to divide two tensors. 

## Model (Computation Graph)
Once the data has been processed and it is in the proper format, all you need to do now is to define your model. Here is where things begin to change a little as compared to how you would build your neural networks using, say, something like Keras or Tensorflow. However, you will realize quickly as you go along that PyTorch doesn't differ much from other deep learning tools. At the end of the day we are constructing a computation graph, which is used to dictate how data should flow and what type of operations are performed on this information. 

For illustration purposes, we are building the following neural network or computation graph:


![alt text](https://drive.google.com/uc?export=view&id=1l-sKpcCJCEUJV1BlAqcVAvLXLpYCInV6)

In [5]:
class Neural_Network(nn.Module):
    def __init__(self, ):
        super(Neural_Network, self).__init__()
        # 决定因素
        # TODO: 参数可以参数化，而不是在此处声明
        self.inputSize = 2
        self.outputSize = 1
        self.hiddenSize = 3
        
        # 权重
        self.W1 = torch.randn(self.inputSize, self.hiddenSize) #产生3X2随机数组成的数组
        self.W2 = torch.randn(self.hiddenSize, self.outputSize) #这里是3X1
        
    def forward(self, X):#通过网络向前传播
        self.z = torch.matmul(X, self.W1) # 3 X 3 ".dot"不在PyTorch中广播
        self.z2 = self.sigmoid(self.z) #激活函数
        self.z3 = torch.matmul(self.z2, self.W2)#z2，W2矩阵相乘
        o = self.sigmoid(self.z3) #最终激活函数
        return o
        
    def sigmoid(self, s):#神经网络的激活函数
        return 1 / (1 + torch.exp(-s))#一个能够产生输出的（未经训练的）神经网络
    
    def sigmoidPrime(self, s):#S型函数的导数
        return s * (1 - s)
    
    def backward(self, X, y, o):#通过网络向后传播
        self.o_error = y - o #输出错误
        self.o_delta = self.o_error * self.sigmoidPrime(o) #误差的导数
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))#我们的隐藏层权重在多大程度上导致了输出误差
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)#z2误差导数
        self.W1 += torch.matmul(torch.t(X), self.z2_delta)#调整第一组权重
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)#调整第二组权重
        
    def train(self, X, y):
        # 向前+向后计算训练
        o = self.forward(X)#通过启动前向传播来定义我们的输出
        self.backward(X, y, o)#通过在函数中调用后向函数来启动它
        
    def saveWeights(self, model):
        # 我们将使用PyTorch内部存储功能
        torch.save(model, "NN")
        # 您可以使用以下方法重新加载具有所有权重的模型，依此类推：
        # torch.load("NN")
        
    def predict(self):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(xPredicted))
        print ("Output: \n" + str(self.forward(xPredicted)))
        

For the purpose of this tutorial, we are not going to be talking math stuff, that's for another day. I just want you to get a gist of what it takes to build a neural network from scratch using PyTorch. Let's break down the model which was declared via the class above. 

## Class Header
First, we defined our model via a class because that is the recommended way to build the computation graph. The class header contains the name of the class `Neural Network` and the parameter `nn.Module` which basically indicates that we are defining our own neural network. 

```python
class Neural_Network(nn.Module):
```

## Initialization
The next step is to define the initializations ( `def __init__(self,)`) that will be performed upon creating an instance of the customized neural network. You can declare the parameters of your model here, but typically, you would declare the structure of your network in this section -- the size of the hidden layers and so forth. Since we are building the neural network from scratch, we explicitly declared the size of the weights matrices: one that stores the parameters from the input to hidden layer; and one that stores the parameter from the hidden to output layer. Both weight matrices are initialized with values randomly chosen from a normal distribution via `torch.randn(...)`. Note that we are not using bias just to keep things as simple as possible.  

```python
def __init__(self, ):
    super(Neural_Network, self).__init__()
    # parameters
    # TODO: parameters can be parameterized instead of declaring them here
    self.inputSize = 2
    self.outputSize = 1
    self.hiddenSize = 3

    # weights
    self.W1 = torch.randn(self.inputSize, self.hiddenSize) # 3 X 2 tensor
    self.W2 = torch.randn(self.hiddenSize, self.outputSize) # 3 X 1 tensor
```

## The Forward Function
The `forward` function is where all the magic happens (see below). This is where the data enters and is fed into the computation graph (i.e., the neural network structure we have built). Since we are building a simple neural network with one hidden layer, our forward function looks very simple:

```python
def forward(self, X):
    self.z = torch.matmul(X, self.W1) 
    self.z2 = self.sigmoid(self.z) # activation function
    self.z3 = torch.matmul(self.z2, self.W2)
    o = self.sigmoid(self.z3) # final activation function
    return o
```

The `forward` function above takes the input `X`and then performs a matrix multiplication (`torch.matmul(...)`) with the first weight matrix `self.W1`. Then the result is applied an activation function, `sigmoid`. The resulting matrix of the activation is then multiplied with the second weight matrix `self.W2`. Then another activation if performed, which renders the output of the neural network or computation graph. The process I described above is simply what's known as a `feedforward pass`. In order for the weights to optimize when training, we need a backpropagation algorithm. 

## The Backward Function
The `backward` function contains the backpropagation algorithm, where the goal is to essentially minimize the loss with respect to our weights. In other words, the weights need to be updated in such  a way that the loss decreases while the neural network is training (well, that is what we hope for). All this magic is possible with the gradient descent algorithm which is declared in the `backward` function. Take a minute or two to inspect what is happening in the code below:

```python
def backward(self, X, y, o):
    self.o_error = y - o # error in output
    self.o_delta = self.o_error * self.sigmoidPrime(o) 
    self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
    self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
    self.W1 += torch.matmul(torch.t(X), self.z2_delta)
    self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
```

Notice that we are performing a lot of matrix multiplications along with the transpose operations via the `torch.matmul(...)` and `torch.t(...)` operations, respectively. The rest is simply gradient descent -- there is nothing to it.

## Training
All that is left now is to train the neural network. First we create an instance of the computation graph we have just built:

```python
NN = Neural_Network()
```

Then we train the model for `1000` rounds. Notice that in PyTorch `NN(X)` automatically calls the `forward` function so there is no need to explicitly call `NN.forward(X)`. 

After we have obtained the predicted output for ever round of training, we compute the loss, with the following code:

```python
torch.mean((y - NN(X))**2).detach().item()
```

The next step is to start the training (foward + backward) via `NN.train(X, y)`. After we have trained the neural network, we can store the model and output the predicted value of the single instance we declared in the beginning, `xPredicted`.  

Let's train!

In [7]:
NN = Neural_Network()#载入神经网络模型
for i in range(1000):#训练神经网络模型1000次
    if (i % 100) == 0:
        print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))#打印均方误差
    NN.train(X, y)#训练模型
#NN.saveWeights(NN) # 保存模型权重

NN.predict()#模型预测

print("Finished training!")#训练结束

#0 Loss: 0.23507122695446014
#100 Loss: 0.003401519963517785
#200 Loss: 0.0031734900549054146
#300 Loss: 0.0030537480488419533
#400 Loss: 0.0029342835769057274
#500 Loss: 0.0028073659632354975
#600 Loss: 0.0026717206928879023
#700 Loss: 0.0025271554477512836
#800 Loss: 0.002374356146901846
#900 Loss: 0.0022152026649564505
Predicted data based on trained weights: 
Input (scaled): 
tensor([0.5000, 1.0000])
Output: 
tensor([0.9335])
Finished training!


The loss keeps decreasing, which means that the neural network is learning something. That's it. Congratulations! You have just learned how to create and train a neural network from scratch using PyTorch. There are so many things you can do with the shallow network we have just implemented. You can add more hidden layers or try to incorporate the bias terms for practice. I would love to see what you will build from here. Reach me out on [Twitter](https://twitter.com/omarsar0) if you have any further questions or leave your comments here. Until next time!

## References:
- [PyTorch nn. Modules](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html#pytorch-custom-nn-modules)
- [Build a Neural Network with Numpy](https://enlight.nyc/neural-network)